In [8]:
# 导入需要的Package并设定种子
suppressMessages(suppressWarnings(library(tidyverse)))
suppressWarnings(suppressMessages(library(Runuran)))
suppressWarnings(suppressMessages(library(mvtnorm)))
suppressWarnings(suppressMessages(library(e1071)))
suppressWarnings(suppressMessages(library(Bessel)))
set.seed(20250402)

In [9]:
# 生成IGN分布的随机数
rIGN <- function(n, Mu, Sigma, Lambda) {
  
  # 生成τ分布的随机数
  distr <- udig(mu = Lambda / (Lambda - 1), lambda = Lambda)
  gen <- pinvd.new(distr)
  tau <- ur(gen, n)
  
  # 随机表示方法生称多元GeN分布随机数
  z <- rmvnorm(n, rep(0, nrow(Sigma)), Sigma)
  x <- matrix(rep(Mu,n), nrow = n, byrow = TRUE) + z / sqrt(tau)
  return(x)
}

# 定义IGN分布的对数似然函数
log_f1 <- function(X, Omega, B, sigma, lambda) {
  d <- nrow(X)
  f <- function(data) {
    x <- data[1:d]
    omega <- data[-(1:d)]
    alpha <- t(x - t(B) %*% omega) %*% solve(sigma) %*% (x - t(B) %*% omega) * lambda + (lambda - 1)^2
    result <- sqrt(lambda) * exp(lambda - 1) / 2^((d-1)/2) / pi^((d+1)/2) / det(sigma)^(1/2) * besselK(sqrt(alpha), 0, expo = FALSE)
    return(result)
  }
  pdf <- apply(rbind(X, Omega), 2, f)
  return(sum(log(pdf)))
}

# IGN分布均值回归的NEM算法
IGN.regression.NEM <- function(B, sigma, lambda, X, Omega, loc = NULL, ep = 1e-4) {
  index <- 0
  k <- 0
  
  n <- ncol(X)
  d <- nrow(X)
  
  f1 <- function(X, Omega, B, sigma, lambda) {
    
    compute.alpha <- function(data) {
      x <- data[1:d]
      omega <- data[-(1:d)]
      return(t(x - t(B) %*% omega) %*% solve(sigma) %*% (x - t(B) %*% omega) + (lambda - 1)^2 / lambda)
    }
    alpha <- apply(rbind(X, Omega), 2, compute.alpha)
    beta <- lambda
    
    result1 <- BesselK(sqrt(alpha * beta), 1, expo = TRUE) / BesselK(sqrt(alpha * beta), 0, expo = TRUE) * (beta / alpha)^0.5
    result2 <- BesselK(sqrt(alpha * beta), -1, expo = TRUE) / BesselK(sqrt(alpha * beta), 0, expo = TRUE) * (beta / alpha)^(-0.5)
    return(cbind(result1, result2))
  }
  
  B_new <- B
  B_new[loc] <- 0
  sigma_new <- sigma
  lambda_new <- lambda
  
  while (k <= 1000) {
    B <- B_new
    sigma <- sigma_new
    lambda <- lambda_new
    
    loglikeli <- log_f1(X, Omega, B, sigma, lambda)
    
    a1 <- f1(X, Omega, B, sigma, lambda)[, 1]
    A1 <- diag(a1)
    B_new <- solve(Omega %*% A1 %*% t(Omega)) %*% (Omega %*% A1 %*% t(X))
    B_new[loc] <- 0
    # cat("Beta:", B_new, "\n")
    
    sigma_new <- (X - t(B_new) %*% Omega) %*% A1 %*% t(X - t(B_new) %*% Omega) / n
    # cat("Sigma:", sigma_new, "\n")
    Ba1 <- mean(a1)
    Bd1 <- mean(f1(X, Omega, B, sigma, lambda)[,2])
    lambda_new <- (-1-sqrt(1-4*Ba1*(2-Ba1-Bd1))) / (2*(2-Ba1-Bd1))
    # cat("Lambda:", lambda_new, "\n")
    
    loglikeli_new <- log_f1(X, Omega, B_new, sigma_new, lambda_new)
    # cat("Loglikelihood:", loglikeli_new, "\n")
    
    if (abs((loglikeli_new - loglikeli) / loglikeli) < ep) {
      index <- 1
      break
    }
    # cat("Iteration number:", k, "\n")
    k <- k + 1
  }
  
  result <- list(Beta = B_new,
                 Sigma = sigma_new,
                 Lambda = lambda_new,
                 Kurtosis = 3 * (lambda_new + 1) / lambda_new^2, 
                 Loglikelihood = loglikeli_new,
                 number = k,
                 index = index)
  return(result)
}

# 设置参数并生成样本
size <- 500
B0 <- matrix(c(0, 0.5, -0.75, 0.25, 0, -1), ncol = 2)
Sigma0 <- matrix(c(1, 0.5, 0.5, 1), nrow = 2)
Lambda0 <- 9
Omega <- rbind(rep(5, size), runif(size, -1, 1), rgamma(size, 2, 3))

Mu <- t(B0) %*% Omega
X <- apply(Mu, 2, function(mu) rIGN(n = 1, Mu = mu, Sigma = Sigma0, Lambda = Lambda0))

# 矩估计确定初值
B <- matrix(1, nrow = nrow(B0), ncol = ncol(B0))
s <- var(t(X))
kur <- apply(X, 1, kurtosis, type = 1) %>% mean()
l <- (3+sqrt(12*kur+9)) / (2*kur)

# 进行极大似然估计
IGN.regression.NEM(B, s, l, X, Omega)

0.01472069,0.24639491
0.37135396,0.00126918
-0.80075572,-0.93763758
0.9699951,0.4337185
0.4337185,0.9579557


In [10]:
# 生成GaN分布的随机数
rGaN <- function(n, Mu, Sigma, Lambda) {
  # 生成τ分布的随机数
  tau <- rgamma(n, Lambda, Lambda - 1)
  
  # 随机表示方法生称多元GeN分布随机数
  z <- rmvnorm(n, rep(0, nrow(Sigma)), Sigma)
  x <- matrix(rep(Mu,n), nrow = n, byrow = TRUE) + z / sqrt(tau)
  return(x)
}

# 定义GaN分布的对数似然函数
log_f2 <- function(X, Omega, B, sigma, lambda) {
  d <- nrow(X)
  f <- function(data) {
    x <- data[1:d]
    omega <- data[-(1:d)]
    phi <- t(x - t(B) %*% omega) %*% solve(sigma) %*% (x - t(B) %*% omega) / 2
    result <- gamma(lambda + 0.5) * (lambda - 1)^lambda / (2 * pi)^(d / 2) / det(sigma)^0.5 / gamma(lambda) / (lambda - 1 + phi)^(lambda + 0.5)
    return(result)
  }
  pdf <- apply(rbind(X, Omega), 2, f)
  return(sum(log(pdf)))
}

# GaN分布均值回归的NEM算法
GaN.regression.NEM <- function(B, sigma, lambda, X, Omega, loc = NULL, ep = 1e-4) {
  index <- 0
  k <- 0
  
  n <- ncol(X)
  d <- nrow(X)
  
  
  
  US_Ga <- function(lambda, c) {
    index <- 0
    k <- 0
    
    lambda_new <- lambda
    
    while (index == 0) {
      lambda <- lambda_new
      q <- c + log(lambda - 1) + 1 / (lambda - 1) - digamma(lambda) + pi^2 * lambda / 6 - 1 / lambda
      lambda_new <- (q + sqrt(q^2 + 2 * pi^2 / 3)) / (pi^2 / 3)
      
      if (abs(lambda_new - lambda) < 1e-5) {
        index <- 1
        break
      }
      k <- k + 1
    }
    return(lambda_new)
  }
  
  f2 <- function(X, Omega, B, sigma, lambda) {
    
    alpha <- lambda + 0.5
    compute.beta <- function(data) {
      x <- data[1:d]
      omega <- data[-(1:d)]
      return(lambda - 1 + t(x - t(B) %*% omega) %*% solve(sigma) %*% (x - t(B) %*% omega) / 2)
    }
    beta <- apply(rbind(X, Omega), 2, compute.beta)
    
    result1 <- alpha / beta
    result2 <- digamma(alpha) - log(beta)
    
    return(cbind(result1, result2))
  }
  
  B_new <- B
  B_new[loc] <- 0
  sigma_new <- sigma
  lambda_new <- lambda
  
  while (k <= 1000) {
    B <- B_new
    sigma <- sigma_new
    lambda <- lambda_new
    
    loglikeli <- log_f2(X, Omega, B, sigma, lambda)
    
    a2 <- f2(X, Omega, B, sigma, lambda)[, 1]
    A2 <- diag(a2)
    B_new <- solve(Omega %*% A2 %*% t(Omega)) %*% (Omega %*% A2 %*% t(X))
    B_new[loc] <- 0
    # cat("Beta:", B_new, "\n")
    
    sigma_new <- (X - t(B_new) %*% Omega) %*% A2 %*% t(X - t(B_new) %*% Omega) / n
    # cat("Sigma:", sigma_new, "\n")
    b2 <- f2(X, Omega, B, sigma, lambda)[, 2]
    c <- mean(b2) - mean(a2) + 1
    lambda_new <- US_Ga(lambda, c)
    # cat("Lambda:", lambda_new, "\n")
    
    loglikeli_new <- log_f2(X, Omega, B_new, sigma_new, lambda_new)
    # cat("Loglikelihood:", loglikeli_new, "\n")
    
    if (abs((loglikeli_new - loglikeli) / loglikeli) < ep) {
      index <- 1
      break
    }
    # cat("Iteration number:", k, "\n")
    k <- k + 1
  }
  
  result <- list(Beta = B_new,
                 Sigma = sigma_new,
                 Lambda = lambda_new,
                 Kurtosis = 3 / (lambda_new - 2), 
                 Loglikelihood = loglikeli_new,
                 number = k,
                 index = index)
  return(result)
}

# 设置参数并生成样本
size <- 500
B0 <- matrix(c(0, 0.5, -0.75, 0.25, 0, -1), ncol = 2)
Sigma0 <- matrix(c(1, 0.5, 0.5, 1), nrow = 2)
Lambda0 <- 10
Omega <- rbind(rep(5, size), runif(size, -1, 1), rgamma(size, 2, 3))

Mu <- t(B0) %*% Omega
X <- apply(Mu, 2, function(mu) rGaN(n = 1, Mu = mu, Sigma = Sigma0, Lambda = Lambda0))

# 矩估计确定初值
B <- matrix(1, nrow = nrow(B0), ncol = ncol(B0))
s <- var(t(X))
kur <- apply(X, 1, kurtosis, type = 1) %>% mean()
l <- 3 / kur + 2

# 进行极大似然估计
GaN.regression.NEM(B, s, l, X, Omega)

-0.00136893,0.22373306
0.69641922,0.02450665
-0.79709552,-0.93807361
0.9991442,0.5082125
0.5082125,0.9378895


In [11]:
# 生成IGaN分布的随机数
rIGaN <- function(n, Mu, Sigma, Lambda) {
  
  # 生成τ分布的随机数
  random <- rgamma(n, Lambda, Lambda)
  tau <- 1 / random
  
  # 随机表示方法生称多元GeN分布随机数
  z <- rmvnorm(n, rep(0, nrow(Sigma)), Sigma)
  x <- matrix(rep(Mu,n), nrow = n, byrow = TRUE) + z / sqrt(tau)
  return(x)
}

# 定义IGaN分布的对数似然函数
log_f3 <- function(X, Omega, B, sigma, lambda) {
  d <- nrow(X)
  f <- function(data) {
    x <- data[1:d]
    omega <- data[-(1:d)]
    2^(-0.5*lambda+(5-2*d)/4) * lambda^(0.5*lambda+0.25)*besselK(sqrt(2*lambda*t(x-t(B) %*% omega)%*%solve(sigma)%*%(x-t(B) %*% omega)), -lambda+0.5,expo = FALSE) / pi^(d/2) / det(sigma)^0.5 / gamma(lambda) / (t(x-t(B) %*% omega)%*%solve(sigma)%*%(x-t(B) %*% omega))^(-0.5*lambda+0.25)
  }
  pdf <- apply(rbind(X, Omega), 2, f)
  return(sum(log(pdf)))
}

# IGaN分布均值回归的NEM算法
IGaN.regression.NEM <- function(B, sigma, lambda, X, Omega, loc = NULL, ep = 1e-4) {
  index <- 0
  k <- 0
  
  n <- ncol(X)
  d <- nrow(X)
  
  US_IGa <- function(lambda, c) {
    index <- 0
    k <- 0
    
    lambda_new <- lambda
    
    while (index == 0) {
      lambda <- lambda_new
      q <- c + log(lambda) - digamma(lambda) + pi^2 * lambda / 6 - 1 / lambda
      lambda_new <- (q + sqrt(q^2 + 2 * pi^2 / 3)) / (pi^2 / 3)
      
      if (abs(lambda_new - lambda) < 1e-5) {
        index <- 1
        break
      }
      k <- k + 1
    }
    return(lambda_new)
  }
  
  f3 <- function(X, Omega, B, sigma, lambda) {
    
    compute.alpha <- function(data) {
      x <- data[1:d]
      omega <- data[-(1:d)]
      return(t(x - t(B) %*% omega) %*% solve(sigma) %*% (x - t(B) %*% omega))
    }
    alpha <- apply(rbind(X, Omega), 2, compute.alpha)
    beta <- 2 * lambda
    gamma <- 0.5 - lambda
    
    result1 <- BesselK(sqrt(alpha * beta), gamma + 1, expo = TRUE) / BesselK(sqrt(alpha * beta), gamma, expo = TRUE) * (beta / alpha)^0.5
    result2 <- BesselK(sqrt(alpha * beta), gamma - 1, expo = TRUE) / BesselK(sqrt(alpha * beta), gamma, expo = TRUE) * (beta / alpha)^(-0.5)
    ep <- 1e-6
    result3 <- (log(BesselK(sqrt(alpha * beta), gamma + ep, expo = FALSE)) - log(BesselK(sqrt(alpha * beta), gamma, expo = FALSE))) / ep + 0.5 * log(beta / alpha)
    
    return(cbind(result1, result2, result3))
  }
  
  B_new <- B
  B_new[loc] <- 0
  sigma_new <- sigma
  lambda_new <- lambda
  
  while (k <= 1000) {
    B <- B_new
    sigma <- sigma_new
    lambda <- lambda_new
    
    loglikeli <- log_f3(X, Omega, B, sigma, lambda)
    
    a3 <- f3(X, Omega, B, sigma, lambda)[, 1]
    A3 <- diag(a3)
    B_new <- solve(Omega %*% A3 %*% t(Omega)) %*% (Omega %*% A3 %*% t(X))
    B_new[loc] <- 0
    # cat("Beta:", B_new, "\n")
    
    sigma_new <- (X - t(B_new) %*% Omega) %*% A3 %*% t(X - t(B_new) %*% Omega) / n
    # cat("Sigma:", sigma_new, "\n")
    
    b3 <- f3(X, Omega, B, sigma, lambda)[, 3]
    
    d3 <- f3(X, Omega, B, sigma, lambda)[, 2]
    c <- 1 - mean(b3) - mean(d3)
    lambda_new <- US_IGa(lambda, c)
    # cat("Lambda:", lambda_new, "\n")
    
    loglikeli_new <- log_f3(X, Omega, B_new, sigma_new, lambda_new)
    # cat("Loglikelihood:", loglikeli_new, "\n")
    
    if (abs((loglikeli_new - loglikeli) / loglikeli) < ep) {
      index <- 1
      break
    }
    # cat("Iteration number:", k, "\n")
    k <- k + 1
  }
  
  result <- list(Beta = B_new,
                 Sigma = sigma_new,
                 Lambda = lambda_new, 
                 Kurtosis = 3 / lambda_new, 
                 Loglikelihood = loglikeli_new,
                 number = k,
                 index = index)
  return(result)
}

# 设置参数并生成样本
size <- 500
B0 <- matrix(c(0, 0.5, -0.75, 0.25, 0, -1), ncol = 2)
Sigma0 <- matrix(c(1, 0.5, 0.5, 1), nrow = 2)
Lambda0 <- 10
Omega <- rbind(rep(5, size), runif(size, -1, 1), rgamma(size, 2, 3))

Mu <- t(B0) %*% Omega
X <- apply(Mu, 2, function(mu) rIGaN(n = 1, Mu = mu, Sigma = Sigma0, Lambda = Lambda0))

# 矩估计确定初值
B <- matrix(1, nrow = nrow(B0), ncol = ncol(B0))
s <- var(t(X))
kur <- apply(X, 1, kurtosis, type = 1) %>% mean()
l <- 3 / kur

# 进行极大似然估计
IGaN.regression.NEM(B, s, l, X, Omega)

0.008627501,0.27233680
0.431696965,-0.01979606
-0.786296061,-1.12603747
0.9422935,0.4839991
0.4839991,0.9036418
